In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, InputLayer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set seeds for reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

In [ ]:
# Define file path
train_file_path = os.path.join('..', 'data', 'raw', 'Train.csv')

# Load the data
train_data = pd.read_csv(train_file_path)

print("Train data shape:", train_data.shape)

print("\nFirst few rows of the train dataset:")
print(train_data.head())

print("\nTrain dataset info:")
train_data.info()

In [ ]:
# Drop unnecessary columns
train_data.drop(columns=['Unnamed: 0', 'Time', 'Location'], inplace=True, errors='ignore')

# Define input and output columns
input_columns = ['Temp_2m', 'RelHum_2m', 'DP_2m', 'WS_10m',
                 'WS_100m', 'WD_10m', 'WD_100m', 'WG_10m']
output_column = 'Power'

# Split the data into training, validation, and test sets
train_val_data, test_data = train_test_split(train_data, test_size=0.15, random_state=SEED)
val_size_adjusted = 0.15 / (1 - 0.15)  # Adjust validation size relative to the training set
train_data, val_data = train_test_split(train_val_data, test_size=val_size_adjusted, random_state=SEED)

print("\nAfter splitting:")
print("Training set shape:", train_data.shape)
print("Validation set shape:", val_data.shape)
print("Test set shape:", test_data.shape)

# Separate input features and target variable
train_X = train_data[input_columns]
train_y = train_data[[output_column]]

val_X = val_data[input_columns]
val_y = val_data[[output_column]]

test_X = test_data[input_columns]
test_y = test_data[[output_column]]

# Scale the input features
scaler_X = StandardScaler()
train_X_scaled = scaler_X.fit_transform(train_X)
val_X_scaled = scaler_X.transform(val_X)
test_X_scaled = scaler_X.transform(test_X)

# Scale the target variable
scaler_y = StandardScaler()
train_y_scaled = scaler_y.fit_transform(train_y)
val_y_scaled = scaler_y.transform(val_y)
test_y_scaled = scaler_y.transform(test_y)

In [ ]:
# Build the neural network model with increased capacity
model = Sequential([
    InputLayer(input_shape=(train_X_scaled.shape[1],)),
    Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    Dropout(0.2),
    Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    Dropout(0.2),
    Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    Dropout(0.2),
    Dense(1)  # Linear activation for regression
])

# Compile the model with SGD optimizer with momentum and a learning rate scheduler
optimizer = SGD(learning_rate=1e-3, momentum=0.9)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Implement ReduceLROnPlateau learning rate scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=1e-6, verbose=1)

# Add early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10,
                               restore_best_weights=True, verbose=1)

# Train the model for more epochs with progress tracking
history = model.fit(
    train_X_scaled, train_y_scaled,
    validation_data=(val_X_scaled, val_y_scaled),
    epochs=1000,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Plot the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss = model.evaluate(test_X_scaled, test_y_scaled, verbose=0)
print(f"Test Loss (MSE): {test_loss:.4f}")

# Predict on test data
test_predictions_scaled = model.predict(test_X_scaled)
test_predictions = scaler_y.inverse_transform(test_predictions_scaled)
test_actual = scaler_y.inverse_transform(test_y_scaled)

# Compute RMSE
rmse = np.sqrt(mean_squared_error(test_actual, test_predictions))
print(f"Test RMSE: {rmse:.4f}")

In [ ]:
# Flattened weight arrays helper functions
def get_weights_flat(model):
    # Get model weights and flatten them into a 1D array
    weights = model.get_weights()
    flat_weights = np.concatenate([w.flatten() for w in weights])
    return flat_weights

def set_weights_from_flat(model, flat_weights):
    # Set model weights from a flat 1D array
    weights = []
    idx = 0
    for layer in model.layers:
        for w in layer.get_weights():
            shape = w.shape
            size = np.prod(shape)
            new_w = flat_weights[idx:idx+size].reshape(shape)
            weights.append(new_w)
            idx += size
    model.set_weights(weights)

# Simplified neural network architecture
def create_model():
    model = Sequential([
        Dense(256, activation='relu', input_shape=(train_X_scaled.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1)  # Linear activation for regression
    ])
    return model

# Define the fitness function
def compute_fitness(E_Wi):
    # FI(W_i) = 1 / (E_Wi + epsilon)
    epsilon = 1e-8  # Small constant to prevent division by zero
    FI_Wi = 1 / (E_Wi + epsilon)
    return FI_Wi

# Genetic Algorithm parameters
population_size = 50  # Increased population size
max_generations = 200
mutation_rate = 0.005  # Reduced mutation rate
tolerance = 1e-4  # Tolerance for early stopping
tournament_size = 3  # For tournament selection

# Initialize the base model and population
base_model = create_model()
base_model.build((None, train_X_scaled.shape[1]))

# Get the total number of weights in the model
initial_weights_flat = get_weights_flat(base_model)
num_weights = len(initial_weights_flat)

# Initialize population with random weights
population = []
for _ in range(population_size):
    # Randomly initialize weights
    individual = np.random.randn(num_weights) * 0.1
    population.append(individual)

# Genetic Algorithm loop
best_fitness_overall = -np.inf
best_individual_overall = None
fitness_history = []

for generation in range(max_generations):
    print(f"Generation {generation}")
    # Evaluate fitness for each individual
    fitnesses = []
    for individual in population:
        # Set model weights
        set_weights_from_flat(base_model, individual)
        # Compute loss on training data
        y_pred = base_model.predict(train_X_scaled, verbose=0)
        E_Wi = mean_squared_error(train_y_scaled, y_pred)
        FI_Wi = compute_fitness(E_Wi)
        fitnesses.append(FI_Wi)
    fitnesses = np.array(fitnesses)

    # Keep track of the best individual
    best_fitness_idx = np.argmax(fitnesses)
    best_fitness = fitnesses[best_fitness_idx]
    best_individual = population[best_fitness_idx]
    print(f"Best fitness: {best_fitness:.6f}")
    fitness_history.append(best_fitness)

    # Check for early stopping
    if best_fitness > best_fitness_overall:
        best_fitness_overall = best_fitness
        best_individual_overall = best_individual.copy()
    if 1 / best_fitness_overall < tolerance:
        print("Early stopping criteria met")
        break

    # Tournament selection
    def tournament_selection(population, fitnesses, k):
        selected = []
        for _ in range(len(population)):
            participants = np.random.choice(len(population), k, replace=False)
            best_idx = participants[np.argmax(fitnesses[participants])]
            selected.append(population[best_idx])
        return selected

    selected_population = tournament_selection(population, fitnesses, tournament_size)

    # Generate new population
    new_population = []
    while len(new_population) < population_size:
        # Select parents
        parent1, parent2 = random.sample(selected_population, 2)
        # Uniform crossover
        mask = np.random.rand(num_weights) < 0.5
        offspring1 = np.where(mask, parent1, parent2)
        offspring2 = np.where(mask, parent2, parent1)
        # Mutation
        mutation_mask1 = np.random.rand(num_weights) < mutation_rate
        mutation_values1 = np.random.randn(num_weights) * 0.1
        offspring1[mutation_mask1] += mutation_values1[mutation_mask1]

        mutation_mask2 = np.random.rand(num_weights) < mutation_rate
        mutation_values2 = np.random.randn(num_weights) * 0.1
        offspring2[mutation_mask2] += mutation_values2[mutation_mask2]

        new_population.extend([offspring1, offspring2])

    # Ensure population size does not exceed the limit
    population = new_population[:population_size]

# After GA, set the best individual's weights and evaluate on the test set
set_weights_from_flat(base_model, best_individual_overall)

# Evaluate on test set
y_pred_test = base_model.predict(test_X_scaled)
test_loss = mean_squared_error(test_y_scaled, y_pred_test)
print(f"Test Loss (MSE): {test_loss:.6f}")

# Compute RMSE
test_actual = scaler_y.inverse_transform(test_y_scaled)
test_predictions = scaler_y.inverse_transform(y_pred_test)
rmse = np.sqrt(mean_squared_error(test_actual, test_predictions))
print(f"Test RMSE: {rmse:.4f}")

# Plot fitness over generations
plt.figure(figsize=(10, 6))
plt.plot(fitness_history)
plt.xlabel('Generation')
plt.ylabel('Best Fitness')
plt.title('Best Fitness Over Generations')
plt.grid(True)
plt.show()